In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pltp
import spacy
import lxml.etree as ET
import pandas as pd
import ast

# Cargar el modelo para español
nlp = spacy.load("es_core_news_sm")  # Modelo pequeño en español

In [14]:
def extraer_covariables_lemas(lista_lemas):
    """
    Versión modificada que trabaja directamente con listas de lemas
    """
    tokens = [token.lower() for token in lista_lemas]  # Convertir a minúsculas
    
    with open("recursos/palabras_positivas.txt", "r", encoding="utf-8") as f:
        palabras_positivas = set(word.strip().lower() for word in f.readlines())

    with open("recursos/palabras_negativas.txt", "r", encoding="utf-8") as f:
        palabras_negativas = set(word.strip().lower() for word in f.readlines())

    with open("recursos/palabras_clave.txt", "r", encoding="utf-8") as f:
        palabras_clave = set(word.strip().lower() for word in f.readlines())

    num_pos = sum(token in palabras_positivas for token in tokens)
    num_neg = sum(token in palabras_negativas for token in tokens)
    
    total = len(tokens) if len(tokens) > 0 else 1  # evitar división por 0
    
    # palabras clave específicas
    frecuencia_claves = {}
    for palabra in palabras_clave:
        frecuencia_claves[f"freq_{palabra}"] = tokens.count(palabra)

    palabras_alegria = {'feliz', 'contento', 'alegre', 'satisfecho', 'encantado', 'euforico', 'contenta','satisfecha', 'encantada', 'euforica' }
    palabras_tristeza = {'triste', 'deprimido', 'melancolico', 'desanimado', 'abatido', 'desalentado', 'pesimista', 'tristeza', 'depresion', 'melancolia', 'desanimo', 'desaliento', 'deprimida', 'melancolica', 'desanimada'}
    palabras_enojo = {'enojado', 'furioso', 'irritado', 'molesto', 'indignado', 'enojada', 'furiosa', 'irritada', 'molesta', 'indignada'}
    palabras_miedo = {'miedo', 'temor', 'panico', 'terror', 'ansiedad', 'preocupacion'}
    
    num_alegria = sum(1 for token in tokens if token in palabras_alegria)
    num_tristeza = sum(1 for token in tokens if token in palabras_tristeza)
    num_enojo = sum(1 for token in tokens if token in palabras_enojo)
    num_miedo = sum(1 for token in tokens if token in palabras_miedo)
    
    covariables = {
        "num_palabras_positivas": num_pos,
        "num_palabras_negativas": num_neg,
        "proporcion_palabras_positivas": num_pos / total,
        "proporcion_palabras_negativas": num_neg / total,
        "sentimiento_lexico_total": num_pos - num_neg,
        "num_palabras_alegria": num_alegria,
        "num_palabras_tristeza": num_tristeza,
        "num_palabras_enojo": num_enojo,
        "num_palabras_miedo": num_miedo,
        "proporcion_alegria": num_alegria / total,
        "proporcion_tristeza": num_tristeza / total,
        "proporcion_enojo": num_enojo / total,
        "proporcion_miedo": num_miedo / total,
    }

    # Agregar frecuencias de palabras clave
    covariables.update(frecuencia_claves)
    
    return covariables

In [15]:
def score_sentiment(text):
    scores = []
    for lemma in text:
        if lemma in sentilex:
            scores.append(sentilex[lemma])

    if len(scores) == 0:
        return 0.0
    return sum(scores) / len(scores)

In [16]:
def score_of_docs(docs):
    scores  = []
    for doc in docs:
        scores.append(score_sentiment(doc))
    return scores

In [20]:
if __name__ == "__main__":

    with open('archivos/lemas_por_documento.txt', 'r', encoding='utf-8') as f:
        listas_de_lemmas = [
            ast.literal_eval(line.strip())
            for line in f
            if line.strip()  # descarta líneas vacías
        ]

    tree = ET.parse("recursos/senticon.es.xml")
    root = tree.getroot()
    
    sentilex = {}
    for layer in root.findall("layer"):
        for sign in ("positive", "negative"):
            for lemma in layer.find(sign).findall("lemma"):
                text = lemma.text.strip()
                pol  = float(lemma.get("pol"))
                # solo añadimos si no existe, así tomamos la primera capa
                if text not in sentilex:
                    sentilex[text] = pol
                   
    scores = score_of_docs(listas_de_lemmas)
    print("Min, Max, Mean, Std of scores:", min(scores), max(scores), pd.Series(scores).mean(), pd.Series(scores).std())
    
    # Aplicar la función a cada fila
    lista_covariables = []
    for lista_lemas in listas_de_lemmas:
        covariables = extraer_covariables_lemas(lista_lemas)
        lista_covariables.append(covariables)

    lemmas = [' '.join(lemmas) for lemmas in listas_de_lemmas]
    
    # OPCIÓN 1: Crear un DataFrame combinado desde el inicio
    df_combined = pd.DataFrame({
        'lemmas': lemmas,
        'scores': scores,
    })
    
    # Crear DataFrame de covariables y combinarlo
    df_covariables = pd.DataFrame(lista_covariables)
    
    # Combinar usando concat (por columnas)
    df_final = pd.concat([df_combined, df_covariables], axis=1)
    
    # Guardar todo en un solo CSV
    df_final.to_csv('archivos/sentiment_scores_combined.csv', index=False)

Min, Max, Mean, Std of scores: -0.09930555555555556 0.5707777777777778 0.27393756544904513 0.08824029009832776
